In [1]:
import numpy as np
import scipy.sparse as sparse
import warnings
from numpy.linalg import eigvals
from scipy.sparse.linalg import expm, eigs
from qiskit.opflow import PrimitiveOp, PauliTrotterEvolution
from qiskit.quantum_info import SparsePauliOp, Operator
from qiskit.circuit import Parameter
from qiskit import Aer, execute, QuantumCircuit, transpile

# ignore sparse efficiency warnings
warnings.simplefilter('ignore', sparse.SparseEfficiencyWarning)

# All matrix functions operate with Scipy spare csc as input/return type

def char_to_pauli(char):
    """Returns the Pauli matrix for a given character in I, X, Y, Z"""
    char = char.upper() # convert lower to upper case
    if char == 'I':
        return np.matrix([[1,0],[0,1]])
    if char == 'X':
        return np.matrix([[0,1],[1,0]])
    if char == 'Y':
        return np.matrix([[0,-1j],[1j,0]])
    if char == 'Z':
        return np.matrix([[1,0],[0,-1]])
    raise Exception("Character '" + char + "' does not correspond to Pauli matrix")

def string_to_operator(string):
    """Calculates the operator (Kronecker product) from a Pauli string"""
    op = sparse.csc_matrix(1, dtype=np.complex128)
    for c in string:
        op = sparse.kron(op, char_to_pauli(c))
    return op

def pauli_unitary(string, coefficient):
    """Calculates the unitary matrix exp(-icH) for a given Pauli string H with coefficient c"""
    return expm(-1j * coefficient * string_to_operator(string))

def pauli_circuit(string, coefficient):
    """Quantum circuit for the given Pauli string with coefficient"""
    op = PrimitiveOp(SparsePauliOp(string, coefficient))
    unitary = PauliTrotterEvolution().convert(op.exp_i())
    return unitary.to_circuit().decompose()

def parameterized_pauli_circuit(string, parameter):
    """Quantum circuit for the given Pauli string with coefficient"""
    op = PrimitiveOp(SparsePauliOp(string, 1))
    unitary = PauliTrotterEvolution().convert((parameter*op).exp_i())
    return unitary.to_circuit().decompose()

def circuit_unitary(circuit, simulate = False):
    """Return unitary calculated from Quantum circuit
       if parameter simulate is True: simulate with Aer unitary backend
       default: obtain unitary from qiskit.Operator class
    """
    if simulate:
        backend = Aer.get_backend('unitary_simulator')
        job = execute(circuit, backend)
        result = job.result()
        u = result.get_unitary(circuit)
    else:
        u = Operator(circuit).data
    return sparse.csc_matrix(u, dtype=np.complex128)

def error(a, b, sparse = True):
    """Error between two equal sized unitaries a, b (sparse csc) defined
    as largest magnitude eigenvalue of a-b
    """
    if sparse:
        ev = eigs(a-b, 1, which='LM', return_eigenvectors=False)
        return np.abs(ev[0])
    else:
        return np.max(np.abs(eigvals(a.toarray()-b.toarray())))

class PauliHamiltonian:
    """Class for Hamiltonian built from Pauli strings
    Attributes:
        operators: list of Pauli strings, in order of coefficient size
        coefficients: dictionary {'Pauli String': coefficient}
        qubits: number of qbits 
    """
    
    def __init__(self):
        self.operators = []
        self.coefficients = {}
        self.qubits = 0
        self.__matrix = None
        self.__unitary = None

    def order(self):
        """Return new PauliHamiltonian with operators in order of coefficients"""
        c = dict(self.coefficients) # copy of dictionary
        ops_ordered = []
        while len(c) > 0:
            omin = min(c, key=c.get)
            ops_ordered.append(omin)
            del c['omin']
    
    def matrix(self):
        """Hamiltonian in matrix form"""
        if self.__matrix is None: # matrix form has not been calculated yet
            n = self.dim()
            self.__matrix = sparse.csc_matrix((n, n), dtype=np.complex128)
            for o in self.operators:
                self.__matrix += self.coefficients[o] * string_to_operator(o)
        return self.__matrix
    
    def unitary(self):
        """exp(-iH) in matrix form"""
        if self.__unitary is None: # unitary has not been calculated yet
            self.__unitary = expm(-1j * self.matrix())
        return self.__unitary
    
    def dim(self):
        """Matrix dimension 2^n with n qubits"""
        return 2**self.qubits
    
    def approx_from_unitaries(self, ops = None):
        """Unitary that approximates the full Hamiltonian evolution exp(-iH)
           calculated as matrix product of the single Pauli unitaries for each string:
           U = U1 * U2 ... * Un where Ui are in order as given by ops
           ops: list of Pauli Strings, subset of self.operators
           if ops is None: ops = self.operators
        """
        if ops is None:
            ops = self.operators
        u = sparse.eye(self.dim(), format='csc') # initialize with identity matrix
        for o in ops:
            u = u.dot(pauli_unitary(o, self.coefficients[o]))
        return u
            
    def approx_from_circuit(self, ops = None, **options):
        """Unitary that approximates the full Hamiltonian evolution exp(-iH)
           calculated as a circuit combination of the single Pauli circuits for each string.
           ops: list of Pauli Strings, subset of self.operators
           if ops is None: ops = self.operators
           simulate = True can be given as additional option to simulate with Aer
           (default is to retrieve unitary from Operator.data)
        """
        simulate = False
        if 'simulate' in options:
            simulate = options['simulate']
        if ops is None:
            ops = self.operators    
        if len(ops) < 1:
            return 0
        qc = QuantumCircuit(self.qubits)
        global_phase = 0
        # go through ops in reverse order to built circuit with rightmost unitary first
        for i in range(len(ops)-1,-1,-1):
            if ops[i] == 'I'*self.qubits:
                global_phase += self.coefficients[ops[i]]
            else:
                qc.append(pauli_circuit(ops[i], self.coefficients[ops[i]]),range(self.qubits))
        return np.exp(-1j*global_phase) * circuit_unitary(qc, simulate)
    
    def global_phase(self):
        """Prefactor exp(-ic) of global phase, c = coefficient of III...I"""
        if self.qubits:
            if 'I'*self.qubits in self.coefficients.keys():
                return np.exp(-1j*self.coefficients['I'*self.qubits])
        return 1
    
    def circuit(self, ops = None):
        """Transpiled quantum circuit that approximates the full Hamiltonian evolution exp(-iH)
           calculated as a circuit combination of the single Pauli circuits for each string.
           ops: list of Pauli Strings, subset of self.operators
           if ops is None: ops = self.operators
        """
        if ops is None:
            ops = self.operators    
        if len(ops) < 1:
            return 0
        qc = QuantumCircuit(self.qubits)
        # go through ops in reverse order to built circuit with rightmost unitary first
        for i in range(len(ops)-1,-1,-1):
            if ops[i] != 'I'*self.qubits: # ignore identity (yields global phase)
                qc.append(pauli_circuit(ops[i], self.coefficients[ops[i]]),range(self.qubits))
        return transpile(qc, basis_gates=['u','cx'])
    
    def circuit_error(self, ops = None):
        """Error of circuit()"""
        u = self.global_phase() * circuit_unitary(self.circuit(ops), True)
        return error(u, self.unitary(), False)
    
    def circuit_deviation(self, unitary):
        """Error of this hamiltonians circuit() with respect to a different unitary"""
        u = self.global_phase() * circuit_unitary(self.circuit(), True)
        return error(u, unitary, False)
    
    def save_circuit(self, filename, ops = None):
        qc = self.circuit(ops)
        try:
            file = open(filename, 'x') # read file
        except:
            print("An exception occurred trying to create the file: " + filename) 
        file.write(qc.qasm())
        file.close()
        
    def save_to_file(self, filename):
        """Save to file in format 'coefficient Pauli-string' per line"""
        try:
            file = open(filename, 'x') # read file
        except:
            print("An exception occurred trying to create the file: " + filename) 
        for o in self.operators:
            file.write(str(self.coefficients[o])+" "+o+"\n")
        file.close()
        
    def crop(self, n = 1):
        """Returns a new PauliHamiltonian with the last n (default: 1) Pauli strings omitted"""
        h = PauliHamiltonian()
        h.qubits = self.qubits
        h.operators = list(self.operators[:-n])
        for o in h.operators:
            h.coefficients[o] = self.coefficients[o]
        return h
        
            
def hamiltonian_from_file(filename):
    """Read Hamiltonian from file and return PauliHamiltonian object.
    File format: 'coefficient Pauli-string' per line, separated by space"""
    try:
        file = open(filename) # read file
    except:
        print("An exception occurred trying to read the file: " + filename) 
    lines = file.read().split('\n')
    file.close()
    h = PauliHamiltonian() # create object to return
    for l in lines:
        if len(l): # exclude empty lines
            [c,o] = l.split() # split in coefficient c and operator string o
            if h.qubits:
                if len(o) != h.qubits: # Make sure all operators have the same dimension
                    raise Exception("Operator string '" + o + "' does not match dimension (" + h.qubits + ")")
            else:
                h.qubits = len(o) # set no. qubits to length of operator string
            h.operators.append(o) # add operator to list of operators of object h
            h.coefficients[o] = float(c) # add to the dictionary of coefficients for h
    return h


h = hamiltonian_from_file("hamiltonian_ordered.txt")

In [2]:
# max error of all Pauli unitaries:
m = 0
for o in h.operators:
    c = h.coefficients[o]
    e = error(pauli_unitary(o,c), circuit_unitary(pauli_circuit(o,c)))
    if e > m:
        m = e
print(m)

8.881648663456881e-16


In [2]:
%%time
print("Calculating full unitary")
u = h.unitary()

Calculating full unitary
CPU times: user 13.2 s, sys: 1.54 s, total: 14.8 s
Wall time: 14.7 s


In [5]:
%%time
print("Calculating approximating unitary from unitaries")
u1 = h.approx_from_unitaries()

Calculating approximating unitary from unitaries
CPU times: user 18min 40s, sys: 3.41 s, total: 18min 44s
Wall time: 18min 44s


In [13]:
%%time
print("Calculating approximating unitary from circuit")
u2 = h.approx_from_circuit()

Calculating approximating unitary from circuit
CPU times: user 1min 3s, sys: 24.2 ms, total: 1min 3s
Wall time: 1min 3s


In [14]:
%%time
print("Calculating approximating unitary from circuit (using simulator)")
u3 = h.approx_from_circuit(simulate=True)

Calculating approximating unitary from circuit (using simulator)
CPU times: user 32.3 s, sys: 8.24 s, total: 40.5 s
Wall time: 8.22 s


In [6]:
%%time
error(u,u1)

CPU times: user 246 ms, sys: 188 ms, total: 434 ms
Wall time: 75.5 ms


0.07406384276827156

In [7]:
%%time
error(u,u1,False) # no sparse

CPU times: user 3.61 s, sys: 1.7 s, total: 5.3 s
Wall time: 884 ms


0.07406384276827152

In [15]:
%%time
error(u,u2)

CPU times: user 385 ms, sys: 332 ms, total: 718 ms
Wall time: 123 ms


0.07406384276827217

In [18]:
%%time
error(u,u3)

CPU times: user 2.27 s, sys: 1.71 s, total: 3.98 s
Wall time: 695 ms


0.07406384276826321

In [16]:
%%time
error(u,u2,False) # no sparse

CPU times: user 3.93 s, sys: 1.73 s, total: 5.66 s
Wall time: 944 ms


0.07406384276827183

In [17]:
%%time
error(u,u3,False) # no sparse

CPU times: user 4.43 s, sys: 1.71 s, total: 6.14 s
Wall time: 1.02 s


0.07406384276826339

In [19]:
%%time
# Same for the Hamiltonian chopped at 191:
print("Calculating full unitary")
u = h.unitary()
ops = h.operators[:191]
print("Calculating approximating unitary from circuit")
u2 = h.approx_from_circuit(ops)
print("Calculating approximating unitary from circuit (using simulator)")
u3 = h.approx_from_circuit(ops, simulate=True)
print("Error circuit: ", error(u,u2))
print("Error with simulator: ", error(u,u3))

Calculating full unitary
Calculating approximating unitary from circuit
Calculating approximating unitary from circuit (using simulator)
Error circuit:  0.0984674492173192
Error with simulator:  0.09846744921731607
CPU times: user 1min 15s, sys: 5.09 s, total: 1min 20s
Wall time: 60 s


In [39]:
%%time
print('circuit depth: ',h.circuit().depth())
print('circuit error: ',h.circuit_error())

circuit depth:  2042
circuit error:  0.07406384276822622
CPU times: user 49.4 s, sys: 9.79 s, total: 59.2 s
Wall time: 24.5 s


In [40]:
%%time
ops = h.operators[:191]
print('circuit depth: ',h.circuit(ops).depth())
print('circuit error: ',h.circuit_error(ops))

circuit depth:  1430
circuit error:  0.09846744921729812
CPU times: user 25.3 s, sys: 4.33 s, total: 29.7 s
Wall time: 6.93 s


In [43]:
h.save_circuit("c1430.txt", ops)

In [2]:
from scipy.optimize import minimize
import time

def optimize_circuit(hamiltonian, unitary, n_repeat = 1):
    """Attempts to optimize the circuit error (w.r.t. unitary) by minimizing over all coefficients
    repeat n_repeat times"""
    copt = dict(hamiltonian.coefficients) # takes optimized coefficients
    phase = hamiltonian.global_phase()
    nq = hamiltonian.qubits
    ops = list(hamiltonian.operators)
    u = phase * circuit_unitary(hamiltonian.circuit(), True)
    err = error(u, unitary, False) # initial error
    for repetition in range(n_repeat):
        print("Iteration ", repetition+1, " of ",n_repeat,":")
        for o in ops:
            if o != 'I'*nq: # ignore identity
                qc = QuantumCircuit(nq)
                for i in range(len(ops)-1,-1,-1):
                    if ops[i] == o: # parametrize opertor o
                        param = Parameter('param')
                        qc.append(parameterized_pauli_circuit(ops[i], param),range(nq))
                    elif ops[i] != 'I'*nq: # ignore identity (yields global phase)
                        qc.append(pauli_circuit(ops[i], copt[ops[i]]),range(nq))
                def f(p):
                    u = phase * circuit_unitary(qc.bind_parameters({param: float(p)}), True)
                    return error(u, unitary, False)
                print("  Minimizing coefficient for operator ",o)
                m = minimize(f, copt[o], method='COBYLA', tol=0.001)
                if m.fun < err:
                    copt[o] = m.x
                    err = m.fun
                    print("    New minimum found: ",err)
                    print("    New coefficient:   ",copt[o])
    hret = PauliHamiltonian() # create object to return
    hret.qubits = nq
    hret.operators = ops
    hret.coefficients = copt
    return hret

In [ ]:
hc = h.crop(85)
hopt1 = optimize_circuit(hc, h.unitary())
hopt1.save_to_file("hopt1.txt")

In [5]:
# Test if optimized circuit was saved to file correctly
hopt1 = hamiltonian_from_file("hopt1.txt")
print(hopt1.circuit_deviation(h.unitary()))

0.0907965644083497


In [17]:
hopt1a = hopt1.crop(7)
print(hopt1a.circuit_deviation(h.unitary()))
print(hopt1a.circuit().depth())
hopt1a.save_to_file("hopt1a.txt")
hopt1a.save_circuit("c1380.txt")

0.09748663914198114
1380


In [3]:
# Test if optimized circuit was saved to file correctly
hopt1a = hamiltonian_from_file("hopt1a.txt")
print(hopt1a.circuit_deviation(h.unitary()))

0.09748663914198114


In [4]:
hopt1b = optimize_circuit(hopt1a, h.unitary())
hopt1b.save_to_file("hopt1b.txt")

Iteration  1  of  1 :
  Minimizing coefficient for operator  IIIIIZIIII
  Minimizing coefficient for operator  ZIIIIIIIII
    New minimum found:  0.0973673357912975
    New coefficient:    -0.5733858490654371
  Minimizing coefficient for operator  IIZIIIIIII
  Minimizing coefficient for operator  IIIIIIIZII
  Minimizing coefficient for operator  IIIIIIZIII
  Minimizing coefficient for operator  IZIIIIIIII
  Minimizing coefficient for operator  IIIZIIIIII
    New minimum found:  0.09736470231503114
    New coefficient:    -0.3889177647415215
  Minimizing coefficient for operator  IIIIIIIIZI
  Minimizing coefficient for operator  IIIIIIIIIZ
    New minimum found:  0.09688312915984802
    New coefficient:    -0.29510907158947974
  Minimizing coefficient for operator  IIIIZIIIII
    New minimum found:  0.09644733094120482
    New coefficient:    -0.29915594658947975
  Minimizing coefficient for operator  IIIIZIIIIZ
    New minimum found:  0.09590338896414946
    New coefficient:    0.12552

    New minimum found:  0.09238252072309405
    New coefficient:    0.010996760849734562
  Minimizing coefficient for operator  XZZZXIIIII
  Minimizing coefficient for operator  YZZZYIIIII
  Minimizing coefficient for operator  IIIIIXZZZX
  Minimizing coefficient for operator  IIIIIYZZZY
  Minimizing coefficient for operator  IIIXXIIIII
  Minimizing coefficient for operator  IIIYYIIIII
  Minimizing coefficient for operator  IIIIIIIIXX
  Minimizing coefficient for operator  IIIIIIIIYY
    New minimum found:  0.09236296375046944
    New coefficient:    -0.00291756976241806
  Minimizing coefficient for operator  IXZZXXXIII
  Minimizing coefficient for operator  IYZZYXXIII
  Minimizing coefficient for operator  IXZZXYYIII
  Minimizing coefficient for operator  IYZZYYYIII
  Minimizing coefficient for operator  IIXZXXZXII
    New minimum found:  0.09217717947900678
    New coefficient:    -0.0010672404841914982
  Minimizing coefficient for operator  IIYZYXZXII
  Minimizing coefficient for op

In [6]:
hopt1b = hamiltonian_from_file("hopt1a.txt")
print(hopt1b.circuit_deviation(h.unitary()))
hopt1c = hopt1b.crop()
print(hopt1c.circuit().depth())
print(hopt1c.circuit_deviation(h.unitary()))

0.09748663914198114
1379
0.10013878776104364


In [ ]:
def permutations(iterable):
    """Returns permutations of an iterable, permuting the beginning of the iterable first.
       Does not include the original permutation.
       Based on itertools.permutations()
    """
    pool = tuple(reversed(iterable))
    n = len(pool)
    indices = list(range(n))
    cycles = list(range(n, 0, -1))
    while n:
        for i in reversed(range(n)):
            cycles[i] -= 1
            if cycles[i] == 0:
                indices[i:] = indices[i+1:] + indices[i:i+1]
                cycles[i] = n - i
            else:
                j = cycles[i]
                indices[i], indices[-j] = indices[-j], indices[i]
                yield tuple(pool[i] for i in reversed(indices))
                break
        else:
            return

def find_better_perm(hamiltonian, unitary):
    """Attempts to find an order of operators that yields a smaller error"""
    coeff = dict(hamiltonian.coefficients)
    phase = hamiltonian.global_phase()
    nq = hamiltonian.qubits
    u = phase * circuit_unitary(hamiltonian.circuit(), True)
    err = error(u, unitary, False) # initial error
    print("Initial error:",err)
    for ops in permutations(hamiltonian.operators):
        qc = QuantumCircuit(nq)
        for i in range(len(ops)-1,-1,-1):
            if ops[i] != 'I'*nq: # ignore identity (yields global phase)
                qc.append(pauli_circuit(ops[i], coeff[ops[i]]),range(nq))
        u = phase * circuit_unitary(qc, True)
        e = error(u, unitary, False)
        if e < err - 0.0001:
            print("Found better permutation with error ",e)
            hret = PauliHamiltonian() # create object to return
            hret.qubits = nq
            hret.operators = list(ops)
            hret.coefficients = coeff
            return hret
    print("Finished without finding a better permutation. Returning original Hamiltonian.")
    return hamiltonian

hopt2 = find_better_perm(hopt1b, h.unitary())

Initial error: 0.09748663914198114
